<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/Arima_pilkottu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 0. Esitiedot ja Tavoite (ARIMA)
"""
Otsoniennuste Helsinki - ARIMA-malli

Tavoite:
1. Ladata uusi esikäsitelty data (sis. pilvisyys) Parquet-tiedostosta.
2. Sovittaa ARIMA (AutoRegressive Integrated Moving Average) -malli
   Otsoni [µg/m³] -aikasarjaan.
3. Tehdä ennusteita testijaksolle (seuraavat 24 tuntia).
4. Arvioida mallin suorituskykyä (RMSE, MAE) ja verrata baselineen.

HUOM: Perus-ARIMA-malli käyttää vain kohdemuuttujan (Otsoni) omaa
historiaa ennustamiseen, EI muita säämuuttujia (kuten lämpötila, pilvisyys).
Jos halutaan sisällyttää muita muuttujia, tarvitaan ARIMAX tai SARIMAX -malleja.
"""
print("Osa 0: Esitiedot - OK")

In [ ]:
# @title 1. Tuonnit ja Asetukset (ARIMA) - Kokeillaan AR(1)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import os
import warnings # Varoitusten hallintaan

# Tilastollinen mallinnus ja aikasarja-analyysi
from statsmodels.tsa.stattools import adfuller # Stationaarisuustesti
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf # ACF/PACF-kuvaajat
from statsmodels.tsa.arima.model import ARIMA

# Mahdollisesti hyödyllinen automaattiseen kertalukujen valintaan
try:
    import pmdarima as pm
    AUTO_ARIMA_AVAILABLE = True
except ImportError:
    # print("VAROITUS: Kirjastoa 'pmdarima' ei löytynyt. auto_arima ei ole käytettävissä.")
    AUTO_ARIMA_AVAILABLE = False # Varmistetaan Falseksi

# Arviointimetriikat
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Asetukset kuvaajille
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Estetään turhia varoituksia
warnings.filterwarnings("ignore")

# --- Data-asetukset ---
BASE_GITHUB_URL = 'https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/'
PARQUET_PATH = 'data/processed/processed_Helsinki_O3_Weather_Cloudiness_2024_2025_v3.parquet'
DATA_URL = BASE_GITHUB_URL + PARQUET_PATH
TARGET_COLUMN = 'Otsoni [µg/m³]'

# --- Mallinnusasetukset ---
FORECAST_HORIZON = 24
TEST_SPLIT_RATIO = 0.15
# TEST_START_DATE = '2025-03-15' # Vaihtoehtoinen

# ARIMA-parametrit (p, d, q)
# *** MUUTOS TÄSSÄ: Kokeillaan yksinkertaisempaa AR(1) eli (1, 0, 0) mallia ***
# d=0 päätettiin aiemmin stationaarisuuden perusteella
ARIMA_ORDER = (1, 0, 0) # Testataan p=1, d=0, q=0

# Käytetäänkö pmdarima.auto_arimaa (ei käytössä nyt)
USE_AUTO_ARIMA = False # Asetetaan Falseksi varmuuden vuoksi

print("Osa 1: Tuonnit ja Asetukset - OK (ARIMA_ORDER vaihdettu)")
if AUTO_ARIMA_AVAILABLE and USE_AUTO_ARIMA:
     print("pmdarima löytyi, mutta auto_arima ei ole nyt käytössä (USE_AUTO_ARIMA=False).")
elif not AUTO_ARIMA_AVAILABLE:
     print("pmdarima EI löytynyt, kertaluvut (p,q) pitää määrittää manuaalisesti.")

In [ ]:
# @title 2. Datan Lataus ja Peruskäsittely (ARIMA)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import traceback # Virheiden jäljitykseen

print("--- Aloitetaan Datan Lataus ja Peruskäsittely ---")

# Alustetaan muuttujat Noneksi siltä varalta, että jokin vaihe epäonnistuu
data_series = None
train_data = None
test_data = None

try:
    # 1. Lataa data Parquet-tiedostosta URL:n kautta
    print(f"Ladataan dataa: {DATA_URL}")
    response = requests.get(DATA_URL)
    response.raise_for_status() # Tarkista latausvirheet
    # Lue data suoraan muistiin Parquet-muodosta
    df_processed = pd.read_parquet(io.BytesIO(response.content))
    print(f"Data ladattu, muoto: {df_processed.shape}")
    print(f"Sarakkeet: {df_processed.columns.tolist()}")

    # 2. Varmista DatetimeIndex
    if not isinstance(df_processed.index, pd.DatetimeIndex):
        print("Indeksi ei ole DatetimeIndex. Yritetään muuntaa...")
        try:
            df_processed.index = pd.to_datetime(df_processed.index)
            # Jos aikavyöhykettä ei ole, mutta oletetaan että se on paikallinen
            if df_processed.index.tz is None:
                 print("Asetetaan aikavyöhykkeeksi Europe/Helsinki...")
                 # Käytetään robustimpaa tapaa, joka voi käsitellä DST-ongelmia paremmin
                 df_processed = df_processed.tz_localize('Europe/Helsinki', ambiguous='NaT', nonexistent='NaT')
                 rows_before_nat = len(df_processed)
                 df_processed = df_processed.dropna(axis=0, subset=[df_processed.index.name]) # Poista NaT-rivit
                 if len(df_processed) < rows_before_nat:
                      print(f"Poistettu {rows_before_nat - len(df_processed)} riviä aikavyöhykkeen asetuksen (NaT) vuoksi.")

            # Tai jos se on UTC, muunnetaan
            elif str(df_processed.index.tz).lower() == 'utc':
                 print("Muunnetaan UTC -> Europe/Helsinki...")
                 df_processed = df_processed.tz_convert('Europe/Helsinki')

        except Exception as e:
            print(f"VIRHE: Aikaleimaindeksin käsittely epäonnistui: {e}")
            raise # Nosta virhe, koska indeksi on kriittinen

    df_processed.sort_index(inplace=True)
    print(f"Data aikaväliltä: {df_processed.index.min()} - {df_processed.index.max()}")

    # 3. Valitse vain kohdesarake (Otsoni) ARIMA-mallia varten
    if TARGET_COLUMN not in df_processed.columns:
        print(f"VIRHE: Kohdesaraketta '{TARGET_COLUMN}' ei löytynyt datasta!")
        raise KeyError(f"Sarake '{TARGET_COLUMN}' puuttuu.")

    data_series = df_processed[TARGET_COLUMN].copy()

    # Muunnetaan datatyyppi floatiksi varmuuden vuoksi
    data_series = pd.to_numeric(data_series, errors='coerce')

    # 4. Käsittele mahdolliset puuttuvat arvot kohdesarjassa
    initial_length = len(data_series)
    data_series.dropna(inplace=True)
    removed_na = initial_length - len(data_series)
    if removed_na > 0:
        print(f"Poistettu {removed_na} puuttuvaa arvoa kohdesarjasta.")

    if data_series.empty:
        raise ValueError("Kohdesarake on tyhjä tai sisältää vain puuttuvia arvoja.")

    # Tarkistetaan datan tiheys (oletus tunti)
    try:
        inferred_freq = pd.infer_freq(data_series.index)
        print(f"Päätelty datan tiheys: {inferred_freq}")
        if inferred_freq != 'h' and inferred_freq != 'H':
             print("VAROITUS: Datan tiheys ei ole täsmälleen tunti ('H'). Tämä voi vaikuttaa ARIMA-malliin.")
             # Yritetään asettaa tiheys tunniksi, jos puuttuvia aikaleimoja on vähän
             # data_series = data_series.asfreq('H')
             # print("Yritetty asettaa tiheys tunniksi. Uudet NaN-arvot tulee käsitellä.")
    except Exception as e_freq:
         print(f"VAROITUS: Datan tiheyden päättely epäonnistui: {e_freq}")


    # 5. Jaa data harjoitus- ja testijoukkoihin
    # Käytetään TEST_SPLIT_RATIO -suhdetta lopusta
    split_index = int(len(data_series) * (1 - TEST_SPLIT_RATIO))
    train_data = data_series[:split_index]
    test_data = data_series[split_index:]

    # TAI jos TEST_START_DATE on määritelty osassa 1:
    # if 'TEST_START_DATE' in locals() and TEST_START_DATE is not None:
    #    print(f"Käytetään testijakson aloituspäivää: {TEST_START_DATE}")
    #    try:
    #        test_start_dt = pd.Timestamp(TEST_START_DATE, tz='Europe/Helsinki') # Määritä aikavyöhyke
    #        train_data = data_series[data_series.index < test_start_dt]
    #        test_data = data_series[data_series.index >= test_start_dt]
    #    except Exception as e_date:
    #        print(f"VIRHE testijakson aloituspäivän käytössä: {e_date}. Käytetään ratio-jakoa.")
    #        split_index = int(len(data_series) * (1 - TEST_SPLIT_RATIO))
    #        train_data = data_series[:split_index]
    #        test_data = data_series[split_index:]
    # else: # Käytä ratio-jakoa jos päivää ei määritelty
    #    split_index = int(len(data_series) * (1 - TEST_SPLIT_RATIO))
    #    train_data = data_series[:split_index]
    #    test_data = data_series[split_index:]


    print(f"\nDatan jako:")
    print(f"Harjoitusdata: {len(train_data)} havaintoa ({train_data.index.min()} - {train_data.index.max()})")
    print(f"Testidata:     {len(test_data)} havaintoa ({test_data.index.min()} - {test_data.index.max()})")

    if len(train_data) == 0 or len(test_data) == 0:
         raise ValueError("Harjoitus- tai testidata on tyhjä jaon jälkeen.")

    if len(test_data) < FORECAST_HORIZON:
        print(f"VAROITUS: Testidata ({len(test_data)}) on lyhyempi kuin ennustehorisontti ({FORECAST_HORIZON}). Arviointi voi olla epätäydellinen.")


    # 6. Visualisoi aikasarja
    print("\nVisualisoidaan aikasarja...")
    plt.figure(figsize=(14, 7))
    plt.plot(train_data.index, train_data, label='Harjoitusdata', color='royalblue')
    plt.plot(test_data.index, test_data, label='Testidata', color='darkorange')
    plt.title(f'{TARGET_COLUMN} Aikasarja')
    plt.xlabel('Aika')
    plt.ylabel('Otsoni (µg/m³)')
    plt.legend()
    plt.grid(True, linestyle=':')
    plt.tight_layout()
    plt.show()

    print("\nOsa 2: Datan Lataus ja Peruskäsittely - OK")

except KeyError as e:
     print(f"\nVIRHE: Saraketta ei löytynyt: {e}. Tarkista TARGET_COLUMN.")
     data_series = None; train_data = None; test_data = None
except FileNotFoundError:
     print(f"\nVIRHE: Tiedostoa ei löytynyt paikallisesta polusta eikä lataus URL:sta onnistunut.")
     data_series = None; train_data = None; test_data = None
except requests.exceptions.RequestException as e:
     print(f"\nVIRHE: Datan lataus URL:sta epäonnistui: {e}")
     data_series = None; train_data = None; test_data = None
except ValueError as e:
     print(f"\nVIRHE datan käsittelyssä (ValueError): {e}")
     data_series = None; train_data = None; test_data = None
except Exception as e:
     print(f"\nOdottamaton VIRHE datan latauksessa tai peruskäsittelyssä: {e}")
     traceback.print_exc()
     data_series = None; train_data = None; test_data = None

In [ ]:
# @title 3. Stationaarisuuden Tarkastelu ja Differointi (ARIMA)

from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np # Varmistetaan numpy tuonti

print("--- Aloitetaan Stationaarisuuden Tarkastelu ---")

# Varmistetaan, että train_data on olemassa edellisestä solusta
if 'train_data' in locals() and train_data is not None and not train_data.empty:

    # Funktio ADF-testin suorittamiseen ja tulosten tulkintaan
    def perform_adf_test(series, series_name="Series"):
        """Suorittaa ADF-testin ja tulostaa tulokset selkeästi."""
        print(f"\nSuoritetaan ADF-testi sarjalle: {series_name}")
        # Poista NaN-arvot ennen testiä
        result = adfuller(series.dropna())
        print(f'ADF Statistic: {result[0]:.4f}')
        print(f'p-value: {result[1]:.4f}')
        print('Critical Values:')
        for key, value in result[4].items():
            print(f'\t{key}: {value:.4f}')

        # Tulkinta
        if result[1] <= 0.05:
            print(f"-> Tulos: Hylätään nollahypoteesi (p <= 0.05). Sarja '{series_name}' on todennäköisesti stationaarinen.")
            return True # Stationaarinen
        else:
            print(f"-> Tulos: Ei voida hylätä nollahypoteesia (p > 0.05). Sarja '{series_name}' EI todennäköisesti ole stationaarinen.")
            return False # Ei stationaarinen

    # 1. Testaa alkuperäinen harjoitusdata
    is_stationary_orig = perform_adf_test(train_data, "Alkuperäinen harjoitusdata")
    d = 0 # Alustetaan differoinnin kertaluku
    stationary_train_data = train_data # Oletuksena käytetään alkuperäistä, jos se on stationaarinen

    # 2. Jos alkuperäinen ei ole stationaarinen, kokeile ensimmäistä differenssiä
    if not is_stationary_orig:
        print("\nDifferoidaan sarja (1. kertaluku)...")
        train_data_diff1 = train_data.diff().dropna()
        is_stationary_diff1 = perform_adf_test(train_data_diff1, "1. differenssi")
        if is_stationary_diff1:
            d = 1
            stationary_train_data = train_data_diff1
            print(f"\n-> Differoinnin kertaluvuksi (d) asetetaan: {d}")
        else:
            # 3. Jos ensimmäinen differenssi ei ole stationaarinen, kokeile toista (harvinaisempaa)
            print("\nDifferoidaan sarja (2. kertaluku)...")
            train_data_diff2 = train_data_diff1.diff().dropna()
            is_stationary_diff2 = perform_adf_test(train_data_diff2, "2. differenssi")
            if is_stationary_diff2:
                d = 2
                stationary_train_data = train_data_diff2
                print(f"\n-> Differoinnin kertaluvuksi (d) asetetaan: {d}")
            else:
                print("\nVAROITUS: Sarja ei näytä tulevan stationaariseksi edes toisella differoinnilla.")
                print("Käytetään silti toista differenssiä jatkossa, mutta mallin tulokset voivat olla epäluotettavia.")
                d = 2 # Käytetään arvoa 2, vaikka testi ei menisi läpi
                stationary_train_data = train_data_diff2
    else:
        print(f"\n-> Differoinnin kertaluvuksi (d) asetetaan: {d}")


    # 4. Visualisoi alkuperäinen ja mahdollinen differoitu sarja
    plt.figure(figsize=(14, 8))

    plt.subplot(2, 1, 1)
    plt.plot(train_data.index, train_data, label='Alkuperäinen harjoitusdata')
    plt.title('Alkuperäinen Otsoni Aikasarja (Harjoitusdata)')
    plt.legend()
    plt.grid(True, linestyle=':')

    # Piirrä differoitu sarja vain jos differointia tehtiin
    if d > 0:
        plt.subplot(2, 1, 2)
        plt.plot(stationary_train_data.index, stationary_train_data, label=f'{d}. differenssi', color='orange')
        plt.title(f'{d}. Asteen Differoitu Aikasarja')
        plt.legend()
        plt.grid(True, linestyle=':')
    elif 'stationary_train_data' not in locals(): # Jos d=0, mutta muuttujaa ei jostain syystä asetettu
         stationary_train_data = train_data # Varmistus

    plt.tight_layout()
    plt.show()

    # Tallennetaan differoinnin kertaluku myöhempää käyttöä varten
    ARIMA_ORDER = (ARIMA_ORDER[0], d, ARIMA_ORDER[2]) # Päivitetään d-arvo
    print(f"\nARIMA-mallin differoinnin kertaluku 'd' on nyt: {d}")
    print("Seuraavassa vaiheessa määritetään 'p' ja 'q' ACF/PACF-kuvaajien avulla.")
    print("\nOsa 3: Stationaarisuuden Tarkastelu - OK")

else:
    print("VIRHE: Harjoitusdataa ('train_data') ei löytynyt edellisestä solusta. Aja edellinen solu uudelleen.")
    # Asetetaan d varmuuden vuoksi Noneksi, jos data puuttuu
    d = None
    stationary_train_data = None

In [ ]:
# @title 4. ARIMA Kertalukujen (p, q) Määritys (ACF/PACF)

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

print("--- Määritetään ARIMA kertalukuja p ja q (d=0) ---")

# Varmistetaan, että stationaarinen data on olemassa edellisestä solusta
# Koska d=0, käytämme alkuperäistä harjoitusdataa (joka on tallessa stationary_train_data -muuttujassa)
if 'stationary_train_data' in locals() and stationary_train_data is not None and not stationary_train_data.empty:

    # Asetetaan piirrettävien viiveiden (lags) määrä
    # Esim. 72 näyttää 3 päivän korrelaatiot tunneittain
    n_lags = 72

    print(f"\nPiirretään ACF ja PACF {n_lags} viiveellä stationaariselle harjoitusdatalle...")

    fig, axes = plt.subplots(2, 1, figsize=(12, 8))

    # Autokorrelaatiofunktio (ACF) - auttaa määrittämään q:n
    plot_acf(stationary_train_data.dropna(), lags=n_lags, ax=axes[0], title=f'Autokorrelaatiofunktio (ACF) (d={d})')
    axes[0].grid(True, linestyle=':')

    # Osittaisautokorrelaatiofunktio (PACF) - auttaa määrittämään p:n
    # Käytetään oletusmetodia ('yw' tai 'ywm'), joka on yleensä ok
    plot_pacf(stationary_train_data.dropna(), lags=n_lags, ax=axes[1], title=f'Osittaisautokorrelaatiofunktio (PACF) (d={d})', method='ywm')
    axes[1].grid(True, linestyle=':')

    plt.tight_layout()
    plt.show()

    print("\n--- Kuvaajien tulkintaohjeet ---")
    print("Tarkastele yllä olevia kuvaajia:")
    print("1. PACF-kuvaaja (ylempi): Etsi kohta (lag), jonka jälkeen siniset pylväät pysyvät merkittävyysalueen (sininen/harmaa alue) sisällä.")
    print("   Tämä kohta viittaa mahdolliseen **p**-arvoon (AR-kertaluku). Jos pylväät 'häipyvät' hitaasti, AR-osa on merkittävä.")
    print("2. ACF-kuvaaja (alempi): Etsi kohta (lag), jonka jälkeen pylväät pysyvät merkittävyysalueen sisällä.")
    print("   Tämä kohta viittaa mahdolliseen **q**-arvoon (MA-kertaluku). Jos pylväät 'häipyvät' hitaasti, MA-osa on merkittävä.")
    print("3. Jos molemmat näyttävät häipyvän hitaasti, kokeile pieniä p:n ja q:n arvoja (esim. p=1, q=1 tai p=2, q=1 jne.).")
    print("4. Huomioi mahdolliset vahvat piikit kausiluonteisilla viiveillä (esim. 24, 48). Ne voivat viitata SARIMA-mallin tarpeeseen.")
    print("\nPäättele kuvaajien perusteella sopivat p ja q -arvot ja päivitä ne `ARIMA_ORDER`-muuttujaan seuraavaa vaihetta varten.")
    print(f"(Muista, että d={d} on jo päätetty).")
    # Muistutus nykyisestä placeholder-arvosta
    print(f"Nykyinen ARIMA_ORDER placeholder (ennen p/q päivitystä): {ARIMA_ORDER}")
    print("\nOsa 4: ACF/PACF-analyysi - OK")


else:
    print("VIRHE: Stationaarista harjoitusdataa ('stationary_train_data') ei löytynyt edellisestä solusta.")
    # Estetäänkö jatko? Ehkä parempi antaa jatkaa, mutta varoittaa
    print("VAROITUS: Ei voida piirtää ACF/PACF-kuvaajia. p:n ja q:n määritys jää arvailun varaan.")

In [ ]:
# @title 5. ARIMA-Mallin Sovitus (ARIMA) - Lisätty Index Check & Param Print

from statsmodels.tsa.arima.model import ARIMA
import traceback
import numpy as np # Varmistetaan numpy tuonti

print("--- Aloitetaan ARIMA-mallin sovitus ---")

# Varmistetaan, että harjoitusdata ja kertaluvut ovat olemassa
model_fit = None # Alustetaan Noneksi

if 'train_data' in locals() and train_data is not None and not train_data.empty and \
   'ARIMA_ORDER' in locals() and isinstance(ARIMA_ORDER, tuple) and len(ARIMA_ORDER) == 3:

    p, d, q = ARIMA_ORDER
    print(f"Käytetään ARIMA-kertalukuja: p={p}, d={d}, q={q}")

    # ---> LISÄTTY TARKISTUKSIA TÄHÄN <---
    print(f"\nTarkistetaan train_data ennen sovitusta...")
    print(f"train_data pituus: {len(train_data)}")
    nan_count = train_data.isnull().sum()
    inf_count = np.isinf(train_data).sum()
    dup_index_count = train_data.index.duplicated().sum()
    inferred_freq = pd.infer_freq(train_data.index)
    print(f"NaN-arvoja: {nan_count}")
    print(f"Inf-arvoja: {inf_count}")
    print(f"Duplikaatti-indeksejä: {dup_index_count}")
    print(f"Päätelty indeksi-frekvenssi: {inferred_freq}")

    # Yritetään asettaa frekvenssi H, jos se puuttuu, mutta varoen
    if inferred_freq is None:
        print("VAROITUS: Indeksin frekvenssiä ei voitu päätellä. Yritetään asettaa 'H'...")
        try:
            # Luo kopio, jotta alkuperäinen train_data ei muutu jos asfreq epäonnistuu
            train_data_freq = train_data.asfreq('H')
            # Tarkista tuliko lisää NaN-arvoja
            new_nan = train_data_freq.isnull().sum() - nan_count
            if new_nan > 0:
                print(f"VAROITUS: asfreq('H') lisäsi {new_nan} NaN-arvoa. Harkitse niiden täyttämistä tai älä käytä asfreq.")
                # Tässä vaiheessa emme käytä train_data_freq, jos se lisää NaNneja
            else:
                print("asfreq('H') onnistui lisäämättä NaN-arvoja.")
                # Voit halutessasi käyttää tätä: train_data = train_data_freq
        except Exception as e_freq:
            print(f"VAROITUS: asfreq('H') epäonnistui: {e_freq}")


    if nan_count > 0 or inf_count > 0 or dup_index_count > 0:
         print("VIRHE: Harjoitusdatassa on NaN/Inf/Duplikaatti-indeksi arvoja! Ei voida sovittaa mallia luotettavasti.")
         model_fit = None # Estetään jatko
    else:
        print("Harjoitusdata vaikuttaa olevan kunnossa (ei NaN/Inf/Dup).")
        try:
            # 1. Alusta ARIMA-malli
            # Poistetaan freq-argumentti varmuuden vuoksi
            model = ARIMA(train_data, order=ARIMA_ORDER)

            # 2. Sovita malli harjoitusdataan
            print("Sovitus käynnissä...")
            model_fit = model.fit()
            print("Sovitus valmis.")

            # ---> LISÄTTY TARKISTUS TÄHÄN <---
            print("\n--- Sovitetut Parametrit ---")
            print(model_fit.params)
            print("--------------------------")

            # 3. Tulosta mallin yhteenveto
            print("\n--- Mallin Yhteenveto ---")
            print(model_fit.summary())
            print("------------------------")
            # ... (Tulkintaohjeet pysyvät samoina) ...
            print("\nTulkintaohjeita Yhteenvetoon:")
            print("- coef: Kertoimien arvot...")
            # ... (loput ohjeet) ...

            print("\nOsa 5: ARIMA-mallin sovitus - OK")

        except ValueError as ve:
            print(f"\nVIRHE ARIMA-mallin sovituksessa (ValueError): {ve}")
            traceback.print_exc()
            model_fit = None
        except Exception as e:
            print(f"\nOdottamaton VIRHE ARIMA-mallin sovituksessa: {e}")
            traceback.print_exc()
            model_fit = None

else:
    print("VIRHE: Harjoitusdataa ('train_data') tai ARIMA-kertalukuja ('ARIMA_ORDER') ei löytynyt.")
    model_fit = None

In [ ]:
# @title 6. Ennusteiden Tekeminen (ARIMA) - VAIHDETTU .predict()-metodiin

import pandas as pd
import matplotlib.pyplot as plt
import traceback

print("--- Aloitetaan ennusteiden tekeminen sovitetulla ARIMA-mallilla (käyttäen .predict()) ---")

# Varmistetaan, että sovitettu malli ja testidata ovat olemassa
forecast_values = None # Alustetaan Noneksi

if 'model_fit' in locals() and model_fit is not None and \
   'train_data' in locals() and train_data is not None and \
   'test_data' in locals() and test_data is not None and not test_data.empty:

    try:
        # 1. Määritä ennusteen alku- ja loppuindeksit/askeleet
        n_train = len(train_data)
        n_forecast_steps = len(test_data)
        forecast_start_index = n_train # Ensimmäinen ennustettava indeksi on heti harjoitusdatan jälkeen
        forecast_end_index = n_train + n_forecast_steps - 1 # Viimeinen ennustettava indeksi

        print(f"Tehdään {n_forecast_steps} askeleen ennuste testijaksolle käyttäen .predict(start={forecast_start_index}, end={forecast_end_index})...")


        if n_forecast_steps > 0:
            # 2. *** MUUTOS TÄSSÄ: Käytä .predict() ***
            # Tämä ennustaa arvot määritellylle indeksivälille perustuen malliin
            forecast_mean = model_fit.predict(start=forecast_start_index, end=forecast_end_index)

            # .predict() palauttaa suoraan Pandas Seriesin, jos malli alustettiin Seriesillä
            if isinstance(forecast_mean, pd.Series):
                 forecast_values = forecast_mean
                 # Tarkistetaan indeksi
                 if not forecast_values.index.equals(test_data.index[:n_forecast_steps]):
                      print("VAROITUS: Ennusteen indeksi ei täsmää täysin testidatan indeksiin. Yritetään korjata...")
                      try:
                           forecast_values.index = test_data.index[:n_forecast_steps]
                      except Exception as e_idx:
                           print(f"Ennusteen indeksin korjaus epäonnistui: {e_idx}")
                           # Voitaisiin asettaa forecast_values = None, mutta yritetään jatkaa

                 print(".predict()-ennusteet laskettu.")

                 # 3. Tulosta ennusteiden alku ja loppu
                 print("\nEnnusteiden alku:")
                 print(forecast_values.head())
                 print("\nEnnusteiden loppu:")
                 print(forecast_values.tail())

                 # Tarkistetaan NaN-arvot
                 nan_in_forecast = forecast_values.isnull().sum()
                 if nan_in_forecast > 0:
                      print(f"\nVAROITUS: Ennuste sisältää {nan_in_forecast} NaN-arvoa!")

            else:
                 print(f"VIRHE: .predict() ei palauttanut odotettua Pandas Series -objektia (tyyppi: {type(forecast_mean)})")
                 forecast_values = None

        else:
             print("Testidata tyhjä tai ennusteaskelia nolla, ei voida tehdä ennustetta.")
             forecast_values = None


        print("\nOsa 6: Ennusteiden tekeminen (.predict()) - OK")

    except Exception as e:
        print(f"\nOdottamaton VIRHE ennusteiden tekemisessä (.predict()): {e}")
        traceback.print_exc()
        forecast_values = None

else:
    print("VIRHE: Sovitettua mallia ('model_fit'), harjoitusdataa ('train_data') tai testidataa ('test_data') ei löytynyt.")
    forecast_values = None

In [ ]:
# @title 7. Tulosten Arviointi ja Vertailu (ARIMA) - LISÄTTY O3_THRESHOLD MÄÄRITTELY

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import traceback # Tuotu jo aiemmin, mutta varmistetaan

# --- VARMISTETAAN PARAMETRIEN MÄÄRITTELY TÄSSÄ SOLUSSA ---
# Lisätään puuttuva muuttuja suoraan tähän soluun
O3_THRESHOLD_8H_AVG = 85 # µg/m³ (Kokeillaan pienempää, vaikka virallinen varoitusraja)
# Varmistetaan myös prediction_horizon (vaikka se todennäköisesti onkin jo muistissa)
if 'PREDICTION_HORIZON' not in locals(): PREDICTION_HORIZON = 24
# Varmistetaan ARIMA_ORDER referenssiä varten
if 'ARIMA_ORDER' not in locals(): ARIMA_ORDER = ('?', '?', '?') # Placeholder, jos ei löydy
# ----------------------------------------------------------

print("--- Aloitetaan ARIMA-ennusteiden arviointi ---")

# Varmistetaan ennusteiden ja testidatan olemassaolo
evaluation_possible = False
rmse_arima = None; mae_arima = None
rmse_baseline = None; mae_baseline = None
baseline_forecast = None

if 'forecast_values' in locals() and isinstance(forecast_values, pd.Series) and not forecast_values.empty and \
   'test_data' in locals() and isinstance(test_data, pd.Series) and not test_data.empty:

    if len(forecast_values) == len(test_data):
        print(f"Arvioidaan {len(test_data)} ennustepistettä.")
        evaluation_possible = True
        common_index = test_data.index
        try:
            if not forecast_values.empty:
                forecast_values = forecast_values.reindex(common_index)
                if forecast_values.isnull().any():
                     print("VAROITUS: NaN-arvoja ennusteessa indeksin kohdistuksen jälkeen. Täytetään ffill/bfill.")
                     forecast_values = forecast_values.ffill().bfill()
                     if forecast_values.isnull().any():
                          print("VIRHE: Ei voitu täyttää kaikkia NaN-arvoja ennusteesta.")
                          evaluation_possible = False
                if test_data.isnull().any():
                     print("VAROITUS: test_data sisältää NaN-arvoja ennen arviointia. Yritetään täyttää...")
                     test_data = test_data.ffill().bfill()
                     if test_data.isnull().any():
                          print("VIRHE: Ei voitu täyttää NaN-arvoja testidatasta.")
                          evaluation_possible = False
            else:
                print("VIRHE: 'forecast_values' on tyhjä ennen reindexiä.")
                evaluation_possible = False
        except Exception as e_reindex:
            print(f"VIRHE indeksin kohdistuksessa: {e_reindex}")
            evaluation_possible = False
    else:
        print(f"VIRHE: Ennusteiden ({len(forecast_values)}) ja testidatan ({len(test_data)}) pituudet eivät täsmää.")
else:
    missing_eval_vars = []
    if 'forecast_values' not in locals() or not isinstance(forecast_values, pd.Series) or forecast_values.empty: missing_eval_vars.append('forecast_values')
    if 'test_data' not in locals() or not isinstance(test_data, pd.Series) or test_data.empty: missing_eval_vars.append('test_data')
    print(f"VIRHE: Arviointiin tarvittavia muuttujia puuttuu tai ne ovat tyhjiä: {missing_eval_vars}.")

# --- Metriikoiden laskenta (vain jos evaluation_possible on True) ---
if evaluation_possible:
    try:
        # --- Baseline-laskenta ---
        def calculate_baseline_persistence_arima(targets_series, prediction_horizon):
            """Laskee naiivin persistenssi-baselinen Seriesille."""
            print("Lasketaan Baseline-ennuste (jakson eka arvo toistuu)...")
            # ... (funktion sisältö kuten edellisessä vastauksessa) ...
            try:
                if targets_series.empty: return pd.Series(dtype=float)
                first_val = targets_series.iloc[0]
                baseline_preds_np = np.repeat(first_val, len(targets_series))
                baseline_preds = pd.Series(baseline_preds_np, index=targets_series.index)
                return baseline_preds
            except Exception as e_base:
                print(f"VIRHE baseline-laskennassa: {e_base}")
                return None

        baseline_forecast = calculate_baseline_persistence_arima(test_data, PREDICTION_HORIZON)

        # --- Regressiometriikat ---
        print("\nLasketaan regressiometriikat...")
        rmse_arima = np.sqrt(mean_squared_error(test_data, forecast_values))
        mae_arima = mean_absolute_error(test_data, forecast_values)
        print(f"\n--- ARIMA({ARIMA_ORDER[0]},{ARIMA_ORDER[1]},{ARIMA_ORDER[2]}) Mallin Arviointi ---")
        print(f"RMSE: {rmse_arima:.4f} µg/m³")
        print(f"MAE:  {mae_arima:.4f} µg/m³")

        if baseline_forecast is not None and len(baseline_forecast) == len(test_data):
            if baseline_forecast.isnull().any():
                print("VAROITUS: Baseline-ennuste sisältää NaN-arvoja. Täytetään.")
                baseline_forecast = baseline_forecast.ffill().bfill()
            if not baseline_forecast.isnull().any():
                rmse_baseline = np.sqrt(mean_squared_error(test_data, baseline_forecast))
                mae_baseline = mean_absolute_error(test_data, baseline_forecast)
                print(f"\n--- Baseline-Mallin Arviointi (Naiivi Persistenssi) ---")
                print(f"RMSE: {rmse_baseline:.4f} µg/m³")
                print(f"MAE:  {mae_baseline:.4f} µg/m³")
                # --- Vertailu ---
                print("\n--- Vertailu Baselineen ---")
                if rmse_arima is not None and rmse_baseline is not None:
                    improvement_rmse = rmse_baseline - rmse_arima
                    print(f"ARIMA vs Baseline RMSE: {improvement_rmse:+.4f} µg/m³ ({'ARIMA parempi' if improvement_rmse > 0 else 'Baseline parempi tai sama'})")
                else: print("RMSE-vertailua ei voida tehdä.")
                if mae_arima is not None and mae_baseline is not None:
                    improvement_mae = mae_baseline - mae_arima
                    print(f"ARIMA vs Baseline MAE:  {improvement_mae:+.4f} µg/m³ ({'ARIMA parempi' if improvement_mae > 0 else 'Baseline parempi tai sama'})")
                else: print("MAE-vertailua ei voida tehdä.")
            else: print("\nBaseline-metriikoita ei voitu laskea NaN-arvojen vuoksi.")
        else: print("\nBaseline-ennustetta ei voitu laskea tai sen pituus ei täsmää.")

        # --- 8h Liukuvan keskiarvon arviointi ---
        # Käytetään nyt solun alussa määriteltyä O3_THRESHOLD_8H_AVG
        print(f"\nLasketaan 8h liukuvia keskiarvoja koko testijaksolle ja verrataan kynnysarvoon ({O3_THRESHOLD_8H_AVG} µg/m³)...")
        try:
            actual_series_8h = test_data.rolling(window=8, min_periods=1).mean()
            pred_series_8h = forecast_values.rolling(window=8, min_periods=1).mean()
            actual_warning_hours = actual_series_8h > O3_THRESHOLD_8H_AVG
            pred_warning_hours = pred_series_8h > O3_THRESHOLD_8H_AVG
            n_samples_eval = len(test_data)
            print("\n--- 8h Liukuvan Keskiarvon Varoitustason Ylityksen Arviointi (ARIMA - Tuntitaso) ---")
            print(f"Todellisia varoitustunteja testidatassa (> {O3_THRESHOLD_8H_AVG} µg/m³): {actual_warning_hours.sum()} / {n_samples_eval}")
            print(f"Ennustettuja varoitustunteja (ARIMA):                      {pred_warning_hours.sum()} / {n_samples_eval}")

            if actual_warning_hours.sum() > 0 or pred_warning_hours.sum() > 0:
                print("\nSekaannusmatriisi (Confusion Matrix) varoituksille (ARIMA - Tuntitaso):")
                cm = confusion_matrix(actual_warning_hours, pred_warning_hours, labels=[False, True])
                cm_df = pd.DataFrame(cm, index=['Todellinen EI Varoitusta', 'Todellinen KYLLÄ Varoitus'],
                                   columns=['Ennuste EI', 'Ennuste KYLLÄ'])
                print(cm_df)
                print("\nLuokitteluraportti varoituksille (ARIMA - Tuntitaso):")
                report = classification_report(actual_warning_hours, pred_warning_hours, target_names=['Ei Varoitusta', 'Varoitus'], labels=[False, True], zero_division=0)
                print(report)
                TN, FP, FN, TP = cm.ravel()
                recall_warning = TP / (TP + FN) if (TP + FN) > 0 else 0
                precision_warning = TP / (TP + FP) if (TP + FP) > 0 else 0
                print(f"\n---> TÄRKEIMMÄT VAROITUSMETRIIKAT (Tuntitaso):")
                print(f"  Recall (Herkkyys) 'Varoitus'-luokalle: {recall_warning:.4f}")
                print(f"  Precision (Tarkkuus) 'Varoitus'-luokalle: {precision_warning:.4f}")
            else:
                print("\nEi todellisia eikä ennustettuja varoitustunteja testidatassa kynnysarvolla.")

            print("\nOsa 7: Tulosten Arviointi - OK")

        except Exception as e_8h:
            print(f"VIRHE 8h liukuvan keskiarvon laskennassa tai arvioinnissa: {e_8h}")
            traceback.print_exc()

    except Exception as e:
        print(f"\nVIRHE tulosten arvioinnissa: {e}")
        traceback.print_exc()
else:
     print("\nArviointia ei voida suorittaa, koska ennusteet tai testidata puuttuvat/ovat virheellisiä.")